### Description
Second attempt at machine learning; uses a basic backpropagation network to predict the winner of a game of dota 2, but falls into a local minimum.

In [ ]:
import csv
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score


In [2]:
def read_files():
    global hero, test, train, sample
    hero = pd.read_csv('Data/MatchDetail.csv')
    test = pd.read_csv('Data/MatchOverviewTest.csv')
    train = pd.read_csv('Data/MatchOverviewTraining.csv')
    sample = pd.read_csv('Data/sampleSubmission.csv')
    train.index = train.match_id.tolist()
    del train['match_id']
read_files()
train[(24<train)&(train<108)] -= 1
train[(108 < train) & (train < 114)] -= 2
train.loc[:,'hero_6':'hero_10'] += 111
length = len(train)
X = pd.DataFrame({i:[0 for _ in range(length)] for i in range(1, 223)}, index = train.index)
for index, row in train.iterrows():
    for hero_id in row.tolist()[:10]:
        X.loc[index, hero_id] = 1
y = train.first_5_won.astype('int')


In [ ]:
def sigmoid(x, deriv = False):
    if deriv:
        return x * (1-x)
    else:
        return 1 / (1 + np.exp(-x))
bias = np.ones((X.shape[0], 1))
X = np.hstack((X, bias))
y = y.T
try:
    y = np.array([y.values]).T
    X = X.values
except:
    pass
first_weights = 2 * np.random.random((X.shape[1], 220)) - 1
second_weights = 2 * np.random.random((220, 1)) - 1
high = 6000
learning_rate = 1
momentum = float(input("Momentum?\n"))
first_past_update = 0
second_past_update = 0
for j in range(high + 1):
    first_neurons = sigmoid(np.dot(X,first_weights))
    second_neurons = sigmoid(np.dot(first_neurons,second_weights))
    second_error = y - second_neurons
    if j % 100 == 0:
        print("Error:" + str(np.mean(np.abs(second_error))))
        learning_rate = float(input("Learning rate?\n"))
    second_delta = (second_error) * (sigmoid(second_neurons, deriv = True))
    first_error = (second_delta).dot(second_weights.T)
    first_delta = first_error * (sigmoid(first_neurons, deriv = True))
    first_update = learning_rate * X.T.dot(first_delta) + momentum * first_past_update
    second_update = learning_rate * first_neurons.T.dot(second_delta) + momentum * second_past_update
    first_weights += first_update
    second_weights += second_update
    first_past_update, second_past_update = first_update.copy(), second_update.copy()
    
print("done")

Momentum?
.5
Error:0.56099463448
Learning rate?
1
Error:0.4163
Learning rate?
1
Error:0.4163


In [157]:
second_neurons

array([[ 1.],
       [ 1.],
       [ 1.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]])

In [149]:
np.ones((10, 1))

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]])

In [3]:
np.hstack((a,np.random.random((3,1))))

NameError: name 'a' is not defined